## This notebook contains all steps used to compute the heuristics-based features on training dataset.
- Files that is needed on side to run this code: train_neg_full, train_pos_full, emoticon15.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install tokenizers
!pip install nltk
!pip install pandas
!pip install scipy
!pip install scikit-learn
!pip install ekphrasis

In [ ]:
import numpy as np
import pandas as pd

with open("train_neg_full.txt") as f:
    neg_txt = [line for line in f]
    neg_df = pd.DataFrame(neg_txt, columns=['line']).drop_duplicates(keep='first')
    #print(neg_df.head())
f.close()

with open("train_pos_full.txt") as f:
    pos_txt = [line for line in f]
    pos_df = pd.DataFrame(pos_txt, columns=['line']).drop_duplicates(keep='first')
    #print(pos_df.head())
f.close

pos_df['label'] = 1
neg_df['label'] = -1

full_df_orig= pd.concat([pos_df,neg_df],ignore_index=True)
full_df_orig.tail()

In [ ]:
import pandas as pd
df_emoji = pd.read_csv('emoticon15.csv')

total_emojis =[]

emoji_dictionary = pd.DataFrame(columns=['emoji','meaning', 'sentiment'])

for i in range(len(df_emoji['icons'])):
  str_list_of_emojis = df_emoji['icons'][i]
  str_list_of_emojis = str_list_of_emojis[1:-1]
  #print(str_list_of_emojis)
  list_of_emojis = str_list_of_emojis.split(",")
  #print(list_of_emojis)
  list_of_emojis = [emoji.strip() for emoji in list_of_emojis]
  #print(list_of_emojis)
  list_of_emojis = [emoji[1:-1] for emoji in list_of_emojis]
  #print(list_of_emojis)
  for emoji in list_of_emojis:
    emoji_dictionary.loc[len(emoji_dictionary)] = [emoji, " "+df_emoji['meaning'][i].lower()+" ", df_emoji['Sentiment'][i]]


  total_emojis.extend(list_of_emojis)
print(total_emojis)

In [ ]:
emojis_with_parentheses = []
for emoji in total_emojis:
  if '(' in emoji or ')' in emoji:
    if emoji != ":)" and emoji != ":(" and emoji != ":((" and emoji != ":))":
      emojis_with_parentheses.append(emoji)
print(emojis_with_parentheses)

## Step 0 : Emoji score calculation

In [ ]:
import re
import nltk
from tqdm import tqdm
corpus_orig = []
for i in tqdm(range(len(full_df_orig))):
  review = full_df_orig['line'][i]
  review = review.lower()
  corpus_orig.append(review)


emoji_scores = []
corpus_after_emoji_replacement =[]
for index, line in enumerate(corpus_orig):
    words = set(line.replace("\n","").split(" "))

    emoji_score =0
    present_emoji = words & set(total_emojis)
    if len(present_emoji) > 0:
        for emoji in present_emoji:
          row = emoji_dictionary.loc[emoji_dictionary['emoji'] == emoji]
          replacement = row['meaning'].values[0]
          line = line.replace(emoji, replacement)

          emoji_score+= emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0]

    corpus_after_emoji_replacement.append(line)
    emoji_scores.append(emoji_score)


print(len(emoji_scores))
print(len(corpus_after_emoji_replacement))

In [ ]:
emoji_scores = []

for index, line in enumerate(corpus_orig):
    words = set(line.replace("\n","").split(" "))

    emoji_score =0
    present_emoji = words & set(total_emojis)
    if len(present_emoji) > 0:
        for emoji in present_emoji:
          emoji_score+= emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0]

    for emoji in emojis_with_parentheses:
      replacement = emoji_dictionary[emoji_dictionary['emoji'] == emoji]['sentiment'].values[0]
      line = line.replace(emoji, "")
    emoji_scores.append(emoji_score)


print(len(emoji_scores))

df_with_emoji_scores = full_df_orig.copy()
df_with_emoji_scores.loc[:,'emoji_scores'] = emoji_scores
df_with_emoji_scores.loc[:,'line'] = corpus_after_emoji_replacement

In [ ]:

df_with_emoji_scores.to_csv('ff_train_data_with_emoji_scores.csv')

In [ ]:
def twitter_specific_analysis(line,num_replaced_lines):

  replaced = False
  if "( <user> live on <url>" in line:
    line = line.replace("( <user> live on <url>", "")
    replaced = True

  if "( <user> <url>" in line:
    line = line.replace("( <user> <url>", "")
    replaced = True

  if "( <url>" in line:
    line = line.replace("( <url>","")
    replaced = True
  if "( live at <url>" in line:
    line = line.replace("( live at <url>","")
    replaced = True

  if "( rt <user>" in line:
    line = line.replace("( rt <user>","")
    replaced = True
  """
  if "<url> via <user>" in line:
    line = line.replace("<url> via <user>","")
    replaced = True
  """
  if "( via <user> <url>" in line:
    line = line.replace("( via <user> <url>","")
    replaced = True

  if "( via <user>" in line:
    line = line.replace("( via <user>","")
    replaced = True

  if "< ^ , ^ )" in line:
    line = line.replace("< ^ , ^ )","happy")
    replaced = True
  return line,num_replaced_lines

In [ ]:
def detect_dyck_language(s):
    # Check if the string contains only a single opening parenthesis "("
    if s.endswith("... <url>"):
      # Find the position of the last occurrence of '('
      pos = s.rfind('(')

      # If the character is found, replace it
      if pos != -1:
          s = s[:pos] + '' + s[pos+1:]
      # Replace last opening bracket
      # Also if it contains just one opening bracket - don't count it in
    elif s.count("(") == 1 and s.count(")") == 0:
      return s,None


    stack = []
    for index, char in enumerate(s):
        if char == '(':
            stack.append((char, index))
        elif char == ')':
            if not stack:
                return s,(index, ')')
            stack.pop()

    if stack:
        _, index = stack.pop()
        return s,(index, '(')
    else:
        return s,None

In [ ]:
fake_labels =[]
new_corpus = []
emoji_scores = [score for score in df_with_emoji_scores['emoji_scores']]
new_emoji_scores = []
#STATS
num_replaced_lines=0
pos_emoji_added = 0
neg_emoji_added = 0
num_replaced_lines=0

for i, line in enumerate(corpus_orig):
  emoji_score = emoji_scores[i]

  pred_fake_lables = 0

  line1,num_replaced_lines = twitter_specific_analysis(line,num_replaced_lines)

  # Delete first parenthesis and Insert fake label
  line2,res = detect_dyck_language(line1)

  if res is not None:
    index,char = res

    if char == '(':
      pred_fake_lables = -1
      neg_emoji_added+=1
      emoji_score -= 0.4
      #print(newline)
    else:
      pred_fake_lables = 1
      pos_emoji_added+=1
      emoji_score += 0.4
      #print(newline)

  fake_labels.append(pred_fake_lables)
  new_corpus.append(line2)
  new_emoji_scores.append(emoji_score)


print(f"{len(new_corpus)}")
print(f"{len(fake_labels)}")
print(f"{len(new_emoji_scores)}")
print(f"pos_emoji_added: {pos_emoji_added}")
print(f"neg_emoji_added: {neg_emoji_added}")
print(f"num_replaced_lines: {num_replaced_lines}")

In [ ]:
df_step2_with_fakelables = df_with_emoji_scores.copy()
df_step2_with_fakelables.loc[:,'line'] = new_corpus
df_step2_with_fakelables.loc[:,'emoji_scores'] = new_emoji_scores
df_step2_with_fakelables.loc[:,'fake_lables'] = fake_labels

## Preprocessing before advertisement score calculation

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re

def process_string(input_string:str):
    # Find all hashtags
    hashtags = re.findall(r'<hashtag>(.*?)</hashtag>', input_string)

    # Split the string by hashtags
    parts = re.split(r'<hashtag>.*?</hashtag>', input_string)

    # Remove empty strings from parts
    parts = [part.strip() for part in parts if part.strip()]

    # Format the output list
    hashtags_written_out = input_string.replace('<hashtag> ', '').replace('</hashtag> ', '')
    no_hashtags = ''.join(parts) if '<hashtag>' in input_string else ''
    output = [hashtags_written_out] + [no_hashtags] + [hashtags]

    return output

def preprocess(tweets:list):

    text_processor = TextPreProcessor(
        fix_html=True,
        segmenter="twitter",
        corrector="twitter",
        tokenizer=SocialTokenizer(lowercase=True).tokenize,
        dicts=[emoticons],
        unpack_hashtags=True,
        unpack_contractions=True,
        spell_correct_elong=True,
        normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
        annotate={'hashtag','number'}
    )

    pre_process_1 = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in tweets]
    pre_process_2 = [process_string(tweet) for tweet in pre_process_1]

    return pre_process_2

## Advertisement score calculation

In [ ]:
# Function to count keywords in a line
def count_keywords(line, keywords):
    count = 0
    for keyword in keywords:
        count += line.lower().count(keyword)
    return count

def spam_detection(line):
  listOfElectronics = [" apple "," samsung "," lenovo "," hp "," microsoft "," sony "," logitech ", " asus "," amazon "," dell "," acer " ," intel "," simpletech "," belkin "," gigabyte "]
  book_keywords = [" paperback ", " hardcover "," academic edition "," kindle "," audio cd "," cd-rom "," dvd "," thesaurus edition "," collector's edition ","th edition ","nd edition "]
  electronics_keywords = [" cable "," htmi "," audio "," adapter "," conductor "," gb ", " fiber "," feet "," m "," inch "]

  counts = line.count("<number>")

  counts += sum(char.isdigit() for char in line)

  if any(x in line for x in book_keywords):
    if "<url>" in line:
          return True

  elif " frame " in line and "<url>" in line and counts>0:
      return True

  elif any(x in line for x in listOfElectronics):
      if "<url>" in line and counts>0:
        #print(line)
        return True
  elif "<url>" in line:
      # Initialize counters
      lines_with_keywords_count = 0
      total_keyword_occurrences = 0

      # Iterate through lines and count keywords
      keyword_count = count_keywords(line, electronics_keywords)
      if keyword_count > 0:
          lines_with_keywords_count += 1
          total_keyword_occurrences += keyword_count
      if total_keyword_occurrences>=2:
        return True
      return False
  else:
    return False

In [ ]:
corpus = [line for line in df_step2_with_fakelables['line']]
res = preprocess(corpus)
corpus_with_no_num = []
for triple in res:
  corpus_with_no_num.append(triple[0])
spam_scores = []
for i, line in enumerate(corpus_with_no_num):
    # do spam detection
  score = spam_detection(line)

  if score:
    spam_pred = 1
  else:
    spam_pred = 0
  spam_scores.append(spam_pred)
df_step3_with_spamscore= df_step2_with_fakelables.copy()
df_step3_with_spamscore.loc[:,'spam_score'] = spam_scores
final_corpus = []
corpus_before = [line for line in df_step3_with_spamscore['line']]
for i in range(len(corpus_before)):
  preprocessed_line = corpus_before[i]
  pattern = r'[^a-zA-Z<>.,!? ]'
  # Use re.sub() to replace the matched characters with an empty string
  preprocessed_line = re.sub(pattern, '', preprocessed_line)
  preprocessed_line = preprocessed_line.split()
  preprocessed_line = ' '.join(preprocessed_line)
  final_corpus.append(preprocessed_line)
df_step3_with_spamscore.loc[:,'line'] = final_corpus

## We divided our training set in to subsets, so we can calculate the scores in smaller steps, which was better for us to run on colab

In [ ]:
import pandas as pd
resul_df = df_step3_with_spamscore.copy()
resul_df.head()
from tqdm import tqdm
def divide_into_sets(input_list,num):
    # Calculate the size of each set
    set_size = len(input_list) // num
    remainder = len(input_list) % num

    # Initialize the list of sets
    sets = []
    start = 0

    for i in range(num):
        end = start + set_size + (1 if i < remainder else 0)
        sets.append(input_list[start:end])
        print(f'start {start} and end {end} indices for list {i}')
        start = end

    return sets
corpus= []
count =0
for i in tqdm(range(len(resul_df))):

  review = resul_df['line'][i]
  if isinstance(resul_df['line'][i],float):
    review =""
    count+=1
  corpus.append(review)
divided_list = divide_into_sets(corpus,12)
for j in range(12):
  filename = f"divided_lists_{j}.txt"
  with open(filename, 'w') as file:
        file.write(str(divided_list[j]))

## Step 4 : Emotion score calculation

In [ ]:
!pip install datasets

In [ ]:
def classify_emotion(emotion_scores):
    positive_emotions = {'joy', 'love', 'optimism', 'trust', 'anticipation', 'surprise'}
    negative_emotions = {'anger', 'disgust', 'fear', 'pessimism', 'sadness'}

    # Flatten the list if necessary (assuming input structure)
    if isinstance(emotion_scores, list) and len(emotion_scores) == 1:
        emotion_scores = emotion_scores[0]

    # Find the label with the highest score
    max_score = max(emotion_scores, key=lambda x: x['score'])
    max_label = max_score['label']

    # Classify as positive or negative
    if max_label in positive_emotions:
        classification = 1
    elif max_label in negative_emotions:
        classification = -1

    return max_label, classification
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-emotion-multilabel-latest", return_all_scores=True,device =0)
import json
from datasets import Dataset
import torch
def iterate(j):
    filename = f"divided_lists_{j}.txt"
    with open(filename, 'r') as file:
        corpus_subset = eval(file.read())

    dataset = Dataset.from_dict({"text": corpus_subset})
    emotion_scores = []

    # Define a function to apply the classifier
    def classify_text(batch):
      with torch.no_grad():
        r = pipe(batch["text"])
        return {"results": r}

    # Apply the classifier to the dataset
    results = dataset.map(classify_text, batched=True,batch_size = 64)
    results_list = [row['results'] for row in results]

    emotion_scores = []
    for res in results_list:
        pred_labels = classify_emotion(res)
        emotion_scores.append(pred_labels[1])

    print(len(emotion_scores))
    print(emotion_scores[:100])

    filename = f"emotion_scores_{j}.txt"
    with open(filename, 'w') as file:
        file.write(str(emotion_scores))


In [ ]:
for i in range(12):
  iterate(i)

## Sentiment classification calculation with XLM-T

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import torch
import time
# Preprocess text (username and link placeholders)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()

In [ ]:
from datasets import load_dataset
def calc_sentiment_batch(batch):
    encoded_input = tokenizer(batch['text'], padding=True, return_tensors='pt').to(device)

    with torch.no_grad():
        output = model(**encoded_input)
    scores = output.logits.detach().cpu().numpy()
    sentiments = []

    for score in scores:
        score = softmax(score)
        ranking = np.argsort(score)[::-1]
        firstclass = ranking[0]
        if firstclass == 1:
            firstclass = ranking[1]
        pred = 1 if firstclass == 2 else -1
        sentiments.append(pred)

    batch['sentiment'] = sentiments
    return batch

In [ ]:
import json
def calculate(j):
  from datasets import Dataset
  import time
  # Load the dataset
  filename = f"divided_lists_{j}.txt"
  with open(filename, 'r') as file:
      corpus_subset = json.load(file)

  # Convert the list to a Hugging Face dataset
  dataset = Dataset.from_dict({'text': corpus_subset})

  # Apply the sentiment analysis function
  start_time = time.time()
  dataset = dataset.map(calc_sentiment_batch, batched=True, batch_size=32)
  end_time = time.time()
  print(f"Processing time: {end_time - start_time} seconds")

  # Save the results
  filename = f"sentiment_scores_{j}.txt"
  with open(filename, 'w') as file:
      file.write(str(dataset['sentiment']))

## Vadar predictions for comparison

In [ ]:
!pip install nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
df_step4_with_vadar = df_step3_with_spamscore.copy()
analyzer = SentimentIntensityAnalyzer()
corpus = []
for i in tqdm(range(len(df_step4_with_vadar))):

  review = df_step3_with_spamscore['line'][i]
  if isinstance(df_step4_with_vadar['line'][i],float):
    review =""
  corpus.append(review)

vadars = []
for text in corpus:
  scores = analyzer.polarity_scores(text)
  score = scores['compound']
  vadars.append(score)

print(len(vadars))
df_step4_with_vadar['vadar'] = vadars

## Saving all predictions in one file

In [ ]:
emotion_scores =[]
sentiment_scores =[]
for i in range(12):
    file_path = f"emotion_scores_{i}.txt"
    with open(file_path,"r") as file:
        emotions = eval(file.read())
    #print(len(emotions))
    emotion_scores.extend(emotions)

print(len(emotion_scores))
for i in range(12):
    file_path = f"sentiment_scores_{i}.txt"
    with open(file_path,"r") as file:
        sentiments = eval(file.read())
    #print(len(emotions))
    sentiment_scores.extend(sentiments)

print(len(sentiment_scores))

submission_df = df_step4_with_vadar.copy()

submission_df.loc[:,'emotion_scores'] = emotion_scores
submission_df.loc[:,'sentiment_scores'] = sentiment_scores
submission_df.to_csv('submission_train.csv',index=False)

2270482
2270482
